In [1]:
import torch
import json
from torch.utils.data import random_split, DataLoader
from utils.bert_model import get_dataset, BertFineTuned, tokenize, avgPoolFunc
from transformers import BertTokenizer
torch.manual_seed(0)

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def clean_data():
    with open("data/bigger sample.json", "r") as file:
        with open("data/cleanedData.json", "w") as cleaned:
            for line in file:
                lineJ = json.loads(line)
                if lineJ["label"] in ["Bullish", "Bearish"]:
                    cleaned.write(json.dumps(lineJ)+"\n")

def get_max_length(cleanedDataFile, tokenizer):
    max_length = 0
    with open(cleanedDataFile, "r") as file:
        for line in file:
            lineJ = json.loads(line)
            max_length = max(max_length, len(tokenizer.encode(lineJ["body"])))
    
    return max_length

In [3]:
max_len = get_max_length("data/cleanedData.json", tokenizer)

In [4]:
torch.cuda.is_available()

True

In [5]:
bigData = get_dataset("data/cleanedData.json", max_length=max_len)

In [6]:
dataSize = len(bigData)
trainSize = round(dataSize * 0.9)
testSize = dataSize - trainSize
train, test = random_split(bigData, [trainSize, testSize])

In [7]:
trainSize, testSize

(23784, 2643)

In [9]:
trainloader = DataLoader(train, batch_size = 32, shuffle=True, num_workers=2)
testloader = DataLoader(test, batch_size=32, num_workers=2)
model = BertFineTuned(2, avgPoolFunc).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
criterion = torch.nn.CrossEntropyLoss()

running_loss = 0.0
for i, data in enumerate(trainloader):
    inputs = data["text"].cuda()
    labels = data["label"].cuda()
    
    optimizer.zero_grad()
        
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    running_loss += loss
    
    if i%100 == 99:
        print(f"{running_loss} loss at step {i}")
        run
    
    

65.3664321899414 loss at step 99
128.93601989746094 loss at step 199
194.40985107421875 loss at step 299


KeyboardInterrupt: 

In [ ]:
for param in model.parameters():
    print(param.requires_grad)
